In [1]:
import socket

def check_connectivity(host, port=5432):
    """Test if RDS endpoint is reachable"""
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(10)
        result = sock.connect_ex((host, port))
        sock.close()
        
        if result == 0:
            print(f"✓ SUCCESS: Can reach {host}:{port}")
            return True
        else:
            print(f"✗ FAILED: Cannot connect to {host}:{port}")
            print(f"  Error code: {result}")
            return False
    except socket.gaierror as e:
        print(f"✗ DNS ERROR: Cannot resolve {host}")
        print(f"  {e}")
        return False
    except Exception as e:
        print(f"✗ ERROR: {e}")
        return False

# Replace with your RDS endpoint
check_connectivity("progres-db-ext.cakhbszocobl.us-gov-east-1.rds.amazonaws.com")

✗ FAILED: Cannot connect to progres-db-ext.cakhbszocobl.us-gov-east-1.rds.amazonaws.com:5432
  Error code: 11


False

In [2]:
import boto3
from botocore.exceptions import ClientError

def list_rds_databases():
    """List all RDS database instances in the current AWS account/region"""
    try:
        # Create RDS client
        rds = boto3.client('rds')
        
        # Get all DB instances
        response = rds.describe_db_instances()
        
        print(f"Found {len(response['DBInstances'])} RDS database(s):\n")
        
        for db in response['DBInstances']:
            print(f"Database ID: {db['DBInstanceIdentifier']}")
            print(f"  Engine: {db['Engine']} {db.get('EngineVersion', 'N/A')}")
            print(f"  Status: {db['DBInstanceStatus']}")
            print(f"  Endpoint: {db.get('Endpoint', {}).get('Address', 'N/A')}")
            print(f"  Port: {db.get('Endpoint', {}).get('Port', 'N/A')}")
            print(f"  Instance Class: {db['DBInstanceClass']}")
            print(f"  Storage: {db.get('AllocatedStorage', 'N/A')} GB")
            print("-" * 60)
            
    except ClientError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    list_rds_databases()

Found 3 RDS database(s):

Database ID: aices-keycloak
  Engine: postgres 17.4
  Status: available
  Endpoint: aices-keycloak.cakhbszocobl.us-gov-east-1.rds.amazonaws.com
  Port: 5432
  Instance Class: db.m5.xlarge
  Storage: 400 GB
------------------------------------------------------------
Database ID: progres-db-ext
  Engine: postgres 17.4
  Status: available
  Endpoint: progres-db-ext.cakhbszocobl.us-gov-east-1.rds.amazonaws.com
  Port: 5432
  Instance Class: db.m5.large
  Storage: 400 GB
------------------------------------------------------------
Database ID: progres-db-ext-v1
  Engine: postgres 17.4
  Status: stopped
  Endpoint: progres-db-ext-v1.cakhbszocobl.us-gov-east-1.rds.amazonaws.com
  Port: 5432
  Instance Class: db.m5.xlarge
  Storage: 400 GB
------------------------------------------------------------


In [1]:
import psycopg2

# Connect to database
conn = psycopg2.connect(
    host="<<RDS URL>>",
    port=5432,
    database="test_db",
    user="postgres",
    password="<<PASSWORD>>",
    connect_timeout=10,
        # sslmode="require"  # Important for RDS
)

# Create cursor
cur = conn.cursor()

# Execute query
cur.execute("SELECT version();")
result = cur.fetchone()
print(result)

cur.execute("SELECT current_database()")
result = cur.fetchone()
print(result)
# Close connections
cur.close()

OperationalError: connection to server at "andrewl-test.cakhbszocobl.us-gov-east-1.rds.amazonaws.com" (10.0.16.49), port 5432 failed: timeout expired
